# ch07. 사전 학습 모델(GPT2)
## 03. GPT

### 1) GPT1
- GPT(Generative Pre-training)1
  - 큰 자연어 처리 데이터를 비지도 학습으로 사전 학습한 후, 학습된 가중치로 풀고자 하는 문제를 미세 조정하는 방법론의 모델


- 모델 구조
  - 트랜스포머 모델 사용(버트와 동일)
  - 단, GPT1에서는 트랜스포머의 디코더 구조만 사용(버트는 인코더 구조만 사용)


- 사전 학습
  - 버트와 달리 하나의 사전 학습 방식(전통적 언어 모델 방식) 사용
  - 앞 단어를 활용해 다음 단어를 예측하는 방식으로 사전 학습 진행
  - 별도 라벨이 존재하지 않는 데이터도 학습 가능
    - 비지도 학습으로 분류
  - 많은 데이터로 모델 가중치를 사전 학습할 수 있음
  - 버트와 달리, 실제 문제 대상으로 학습 진행 시에도 언어 모델을 함께 학습
  
input|label
---|---
"< START>"|"나는"
"< START>","나는"|"학교에"
"< START>","나는","학교에"|"간다

### 2) GPT2
- GPT(Generative Pre-training)2
  - OpenAI 제안 , 2018년 발표된 GPT1 모델의 성능을 향상한 모델로서 텍스트 생성에서 특히 좋은 성능을 보임
  
  
- 모델 구조
  - 대부분 GPT1과 동일, 트랜스포머의 디코더를 기반으로 하는 모델
  - 차이점: 레이어 노멀라이제이션이 각 부분 블록의 입력 쪽으로 이동 (기존에는 각 레이어 직후 레지듀얼 커넥션과 함께 적용)


- 학습 데이터 및 모델 크기

| |GPT1|GPT2|
|---|---|---|
|레이어|12개|117만 개|
|가중치|48개|1,542만 개|


### 3) GPT2를 활용한 한국어 언어 생성 모델

- 모델 입력
  - BPE(Byte Pair Encoding) 방식을 사용해 텍스트를 나누어 입력값으로 사용
  - 글자와 문자 사이 적절한 단위를 찾아 나누는 방식으로 높은 성능을 보임
  

In [1]:
import os

import numpy as np
import tensorflow as tf

import gluonnlp as nlp
from gluonnlp.data import SentencepieceTokenizer
from transformers import TFGPT2LMHeadModel

from tensorflow.keras.preprocessing.sequence import pad_sequences

from nltk.tokenize import sent_tokenize

- 라이브러리
  - TFGPT2LMHeadModel: 문장 생성, GPT2 언어 모델을 만들기 위해 생성

- 모델 학습에 필요한 모듈
  - transformers 모듈 중, TFGPT2LMHeadModel
  - gluonnlp의 SentencepieceTokenizer, nlp 모듈

In [2]:
class GPT2Model(tf.keras.Model):
    def __init__(self, dir_path):
        super(GPT2Model, self).__init__()
        self.gpt2 = TFGPT2LMHeadModel.from_pretrained(dir_path)
    
    # vocabulary에 대한 logit 값만 활용하도록 last_hidden_states 출력
    def call(self, inputs):
        return self.gpt2(inputs)[0] 

- 학습된 파라미터 불러오기
  - GPT2의 경우, hugginface에 모델로 등록되어 있지 않아, 파라미터 다운로드가 필요

In [3]:
# gpt_ckpt 폴더 만드는 코드
import wget
import zipfile

wget.download('https://github.com/NLP-kr/tensorflow-ml-nlp-tf2/releases/download/v1.0/gpt_ckpt.zip')

with zipfile.ZipFile('gpt_ckpt.zip') as z:
    z.extractall()

100% [......................................................................] 460908853 / 460908853

- 객체 생성 시 모델 리소스 경로를 인자로 전달하면, 학습된 파라미터를 가지는 GPT2 모델이 형성됨

In [4]:
BASE_MODEL_PATH = './gpt_ckpt'
gpt_model = GPT2Model(BASE_MODEL_PATH)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at ./gpt_ckpt.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


#### ① 사전 학습 모델 문장 생성
- 사전 학습된 GPT2는 언어 모델을 통해 학습   
  → GPT2는 언어 모델이며 텍스트 생성이 가능함

- 사전 학습된 모델을 활용해 언어 생성 결과를 확인하고 성능 알아보기
  - 텍스트를 모델에 입력할 수 있도록 토크나이저 설정 필요
  - SentencepieceTokenizer와 nlb 모듈의 vocab으로 단어 사전과 토크나이저 정의
  
스페셜 토큰|역할
---|---
< unk>|모르는 단어에 대한 토큰
< pad>|배치 데이터 길이 맞추는 용도
< s>|문장 시작을 알림
< /s>|문장 종결을 알림

In [5]:
BATCH_SIZE = 16
NUM_EPOCHS = 10
MAX_LEN = 30
TOKENIZER_PATH = './gpt_ckpt/gpt2_kor_tokenizer.spiece'

tokenizer = SentencepieceTokenizer(TOKENIZER_PATH)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(TOKENIZER_PATH,
                                               mask_token=None,
                                               sep_token=None,
                                               cls_token=None,
                                               unknown_token='<unk>',
                                               padding_token='<pad>',
                                               bos_token='<s>',
                                               eos_token='</s>')

- 생성된 토크나이저 객체와 사전 학습된 GPT2 모델을 활용해 문장 생성 결과 확인하기

- 샘플링 방식
  1. top_k 샘플링 방식
     - top_k 순위 안에 해당하는 어휘만 샘플링하여 어휘를 예측하는 방식
     - top_k 값이 높을수록 무작위 샘플 방식에 가까워지고 낮을수록 탐욕 방식에 가까워짐
     - 각 단어의 확률값을 고려하지 않고 순위만 고려하는 방식으로  
       모델에서 예측한 단어의 확률이 한 단어에 크게 몰려 있다면(Narrow Distribution)  
       확률 낮은 단어가 선택될 수 있어서 생성 결과가 일관되는 좋은 문장을 생성하지 못할 수 있음
  
  2. 뉴클러스 샘플링(Nucleus Sampling)
      - 확률 값의 경계를 top_p로 정하면 확률이 가장 높은 순으로 후보 단어들의 확률을 더했을 때, top_p가 되는 단어 집합에 대해 샘플링
      - 일정 확률 값의 경계를 설정하고 문장을 생성했을 경우, top_k 샘플링보다 안정적으로 개연성 있는 문장을 만들 수 있음

   - 참고: [The Curious Case of Neural Text Degeneration](https://arxiv.org/abs/1904.09751)

In [6]:
def tf_top_k_top_p_filtering(logits, top_k=0, top_p=0.0, filter_value=-99999):
    _logits = logits.numpy()
    top_k = min(top_k, logits.shape[-1])  
    if top_k > 0:
        indices_to_remove = logits < tf.math.top_k(logits, top_k)[0][..., -1, None]
        _logits[indices_to_remove] = filter_value

    if top_p > 0.0:
        sorted_logits = tf.sort(logits, direction='DESCENDING')
        sorted_indices = tf.argsort(logits, direction='DESCENDING')
        cumulative_probs = tf.math.cumsum(tf.nn.softmax(sorted_logits, axis=-1), axis=-1)

        sorted_indices_to_remove = cumulative_probs > top_p
        sorted_indices_to_remove = tf.concat([[False], sorted_indices_to_remove[..., :-1]], axis=0)
        indices_to_remove = sorted_indices[sorted_indices_to_remove].numpy().tolist()
        
        _logits[indices_to_remove] = filter_value
    return tf.constant([_logits])

- 아래 코드 설명 
  - seed_word: 문장 생성의 시작 단어
  - model: 문장 생성 수행
  - max_step: 생성 횟수 제한
  - greedy: 모델 출력 결과에 대해 유연하게 문장 생성을 해줄 수 있는지 선택하게 함
    - Ture: 문장 출력 결과에 대해 가장 확률 높은 단어만 선택
    - False: 출력 단어 가운데 확률 또는 순위 높은 단어만 선택해 무작위로 생성
  - top_k, top_p: greedy가 Flase인 경우에 활용하는 파라미터
    - top_k: 확률 높은 순서대로 k번째까지 높은 단어에 대해 필터링하는 값
    - top_p: 일정 확률값 이상인 단어에 대해 필터링하는 값
    - 두 값이 모두 0이면 필터링하지 않음

In [7]:
def generate_sent(seed_word, model, max_step=100, greedy=False, top_k=0, top_p=0.):

    # 문장 생성 시작할 단어를 문장을 의미하는 변수에 할당하고 토크나이즈함
    sent = seed_word
    toked = tokenizer(sent)
    
    # 문장 생성할 수 있는 반복문에 들어감
    # 토크나이즈된 단어를 인덱스로 변환하고 모델에 입력값으로 넣어 출력값을 받음
    # 모델 출력값에 대하여는 문장 마지막 단어만 선택하게 함
    for _ in range(max_step):
        input_ids = tf.constant([vocab[vocab.bos_token],]  + vocab[toked])[None, :] 
        outputs = model(input_ids)[:, -1, :]
        
        # 단어 선택 방법
        if greedy:
            gen = vocab.to_tokens(tf.argmax(outputs, axis=-1).numpy().tolist()[0])
        else:
            output_logit = tf_top_k_top_p_filtering(outputs[0], top_k=top_k, top_p=top_p)
            gen = vocab.to_tokens(tf.random.categorical(output_logit, 1).numpy().tolist()[0])[0]
        
        # 생성된 텍스트 토큰을 대상으로 문자으이 끝을 알리는 </s> 토큰인지 확인
        # 해당 과정을 max_step만큼 반복하면 문장이 생성됨
        if gen == '</s>':
            break
        sent += gen.replace('▁', ' ')
        toked = tokenizer(sent)

    return sent

- GPT2 같은 생성 모델은 주어진 문장 전제로 최대우도추정(Maximum Likelihood Estimation)을 활용해  
  단어 예측 분포에서 가장 확률 높은 단어를 선택하는 탐욕 검색(Greedy Search)로 새로운 단어를 예측할 수 있음
  - 단, 다음 단어 예측 시 잘못된 단어를 예측하면 학습 상황과 같이  
    다음 단어를 예측할 때, 올바른 단어를 출력할 수 있도록 조정하지 않음  
  → 확률 높은 단어만 선택하여, 생성된 문장에 어색한 문구가 나오거나 반복되는 단어가 발생할 수 있음
  

1. 탐욕(greedy) 방식으로 문장 생성하기
   - 항상 확률이 가장 높은 단어만 선택하기 때문에 모델이 학습한 바이어스(bias)에 따라 일관된 문장만 출력하게 되고,  
     경우에 따라 반복되는 단어가 출력될 수 있음

In [8]:
generate_sent('이때', gpt_model, greedy=True)

'이때부터                                                                                                   '

In [9]:
generate_sent('하루', gpt_model, greedy=True)

'하루'

2. greedy 파라미터를 False로 지정하고 top_k와 top_p를 설정
   - 샘플링 방식을 통해 조금 더 자연스럽고 다양한 문장 확인 가능

In [10]:
generate_sent('이때', gpt_model, top_k=0, top_p=0.95)

'이때문에 목욕하면 와!'

In [11]:
generate_sent('하루', gpt_model, top_k=0, top_p=0.95)

'하루대이가 기습을 가해 오는 바람에 런텍이 수적 우위를 장악(<unk><lf><mv><md><mc>)><mr><md><md><mc><md> <mv><md><mc> <mc>  <mc><md> <md><md> <md>  <mc>   <md>'

#### ② 소설 텍스트 데이터 전처리하기
- 소설 『운수 좋은 날』

In [12]:
DATA_IN_PATH = './data/finetune/'
TRAIN_DATA_FILE = 'finetune_data.txt'

sents = [s[:-1] for s in open(DATA_IN_PATH + TRAIN_DATA_FILE, encoding='utf-8').readlines()]

In [13]:
input_data = []
output_data = []

for s in sents:
    tokens = [vocab[vocab.bos_token],]  + vocab[tokenizer(s)] + [vocab[vocab.eos_token],]
    input_data.append(tokens[:-1])
    output_data.append(tokens[1:])

input_data = pad_sequences(input_data, MAX_LEN, value=vocab[vocab.padding_token])
output_data = pad_sequences(output_data, MAX_LEN, value=vocab[vocab.padding_token])

input_data = np.array(input_data, dtype=np.int64)
output_data = np.array(output_data, dtype=np.int64)

#### ③ 소설 텍스트 미세 조정 모델 학습
- Seq2Seq와 Transformer 모델 사용


- 손실 함수와 정확도 측정 함수
  - loss_object: 크로스 엔트로피로 손실값을 측정하기 위한 객체
  - train_accuracy: 정확도 측정을 위한 객체

In [14]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='accuracy')

# loss 함수: 인자로 정답과 예측한 값을 받아 두 개 값을 비교하여 손실 계산
# real 값 중, vacab[vocab.padding_token]인 값 <PAD>는 손실 계산에서 빼기 위한 함수
def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, vocab[vocab.padding_token]))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

# accuracy 함수: loss 함수와 비슷하나, train_accuracy 함수를 통해 정확도를 체크한다는 점이 다름
def accuracy_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, vocab[vocab.padding_token]))
    mask = tf.expand_dims(tf.cast(mask, dtype=pred.dtype), axis=-1)
    pred *= mask    
    acc = train_accuracy(real, pred)

    return tf.reduce_mean(acc)

In [15]:
gpt_model.compile(loss=loss_function,
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=[accuracy_function])

In [16]:
history = gpt_model.fit(input_data, output_data, 
                    batch_size=BATCH_SIZE, epochs=NUM_EPOCHS,
                    validation_split=0.1)

Epoch 1/10
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.
C:\Users\naeun\anaconda3\lib\site-packages\tensorflow\python\framework\indexed_slices.py:432: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


16/16 [==============================] - ETA: 0s - loss: 4.2235 - accuracy_function: 0.1782

The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


16/16 [==============================] - 88s 6s/step - loss: 4.2235 - accuracy_function: 0.1782 - val_loss: 2.9008 - val_accuracy_function: 0.2147
Epoch 2/10
16/16 [==============================] - 80s 5s/step - loss: 2.9587 - accuracy_function: 0.2385 - val_loss: 2.4976 - val_accuracy_function: 0.2538
Epoch 3/10
16/16 [==============================] - 85s 5s/step - loss: 2.5146 - accuracy_function: 0.2686 - val_loss: 2.3377 - val_accuracy_function: 0.2830
Epoch 4/10
16/16 [==============================] - 87s 5s/step - loss: 2.2447 - accuracy_function: 0.2968 - val_loss: 2.2605 - val_accuracy_function: 0.3059
Epoch 5/10
16/16 [==============================] - 78s 5s/step - loss: 2.0203 - accuracy_function: 0.3173 - val_loss: 2.2218 - val_accuracy_function: 0.3259
Epoch 6/10
16/16 [==============================] - 74s 5s/step - loss: 1.7967 - accuracy_function: 0.3368 - val_loss: 2.2407 - val_accuracy_function: 0.3454
Epoch 7/10
16/16 [==============================] - 73s 5s/step

In [17]:
DATA_OUT_PATH = './output/finetune/'
model_name = "tf2_gpt2_finetuned_model"

save_path = os.path.join(DATA_OUT_PATH, model_name)

if not os.path.exists(save_path):
    os.makedirs(save_path)

gpt_model.gpt2.save_pretrained(save_path)

loaded_gpt_model = GPT2Model(save_path)

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at ./output/finetune/tf2_gpt2_finetuned_model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [18]:
generate_sent('이때', gpt_model, greedy=True)

'이때                                                                                                    '

In [19]:
generate_sent('하루', gpt_model, greedy=True)

'하루에                                                                                                   '

In [20]:
generate_sent('이때', gpt_model, top_k=0, top_p=0.95)

'이때,                                                                                                   '

In [21]:
generate_sent('하루', gpt_model, top_k=0, top_p=0.95)

'하루에게는                                                                                                 '